In [62]:
import torch
import copy
import math
import copy
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [63]:
def to_low(inp_list):
    return [x.lower() for x in inp_list]

In [64]:
def merge(inp_list, bert_list):
    dif = list(set(inp_list).difference(bert_list))
    for x in dif:
        for n, y in enumerate(inp_list):
            if x == y:
                inp_list[n] = '[UNK]'
    return inp_list

In [65]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = """The United States of America is a federal republic consisting of 50 states,
a federal district (Washington, D.C., the capital city of the United States),
five major territories, and various minor islands. The 48 contiguous states and Washington,
D.C., are in central North America between Canada and Mexico; the two other states, Alaska and Hawaii,
are in the northwestern part of North America and an archipelago in the mid-Pacific, respectively,
while the territories are scattered throughout the Pacific Ocean and the Caribbean Sea."""

tokenized_text_bert = tokenizer.tokenize(text)


tokenized_text_inp = ['The', 'United', 'States', 'of', 'America', 'is', 'a', 'federal', 'republic',
                      'consisting', 'of', '50', 'states', ',', 'a', 'federal', 'district', '(',
                      'Washington', ',', 'D.C.', ',', 'the', 'capital', 'city', 'of', 'the', 'United',
                      'States', ')', ',', 'five', 'major', 'territories', ',', 'and', 'various',
                      'minor', 'islands', '.', 'The', '48', 'contiguous', 'states', 'and', 'Washington',
                      ',', 'D.C.', ',', 'are', 'in', 'central', 'North', 'America', 'between', 'Canada',
                      'and', 'Mexico', ';', 'the', 'two', 'other', 'states', ',', 'Alaska', 'and', 'Hawaii',
                      ',', 'are', 'in', 'the', 'northwestern', 'part', 'of', 'North', 'America', 'and',
                      'an', 'archipelago', 'in', 'the', 'mid', '-', 'Pacific', ',', 'respectively', ',',
                      'while', 'the', 'territories', 'are', 'scattered', 'throughout', 'the', 'Pacific',
                      'Ocean', 'and', 'the', 'Caribbean', 'Sea', '.']

tokenized_text_inp = to_low(tokenized_text_inp)
print(tokenized_text_inp, '\n')
print(tokenized_text_bert, '\n')

tokenized_text = merge(tokenized_text_inp, tokenized_text_bert)
print(tokenized_text, '\n')

['the', 'united', 'states', 'of', 'america', 'is', 'a', 'federal', 'republic', 'consisting', 'of', '50', 'states', ',', 'a', 'federal', 'district', '(', 'washington', ',', 'd.c.', ',', 'the', 'capital', 'city', 'of', 'the', 'united', 'states', ')', ',', 'five', 'major', 'territories', ',', 'and', 'various', 'minor', 'islands', '.', 'the', '48', 'contiguous', 'states', 'and', 'washington', ',', 'd.c.', ',', 'are', 'in', 'central', 'north', 'america', 'between', 'canada', 'and', 'mexico', ';', 'the', 'two', 'other', 'states', ',', 'alaska', 'and', 'hawaii', ',', 'are', 'in', 'the', 'northwestern', 'part', 'of', 'north', 'america', 'and', 'an', 'archipelago', 'in', 'the', 'mid', '-', 'pacific', ',', 'respectively', ',', 'while', 'the', 'territories', 'are', 'scattered', 'throughout', 'the', 'pacific', 'ocean', 'and', 'the', 'caribbean', 'sea', '.'] 

['the', 'united', 'states', 'of', 'america', 'is', 'a', 'federal', 'republic', 'consisting', 'of', '50', 'states', ',', 'a', 'federal', 'dis

In [66]:
ner_list = [(0, 4), (11, 11), (18, 18), (20, 20), (26, 28), (31, 31), (41, 41), (45, 45),
            (47, 47), (52, 53), (55, 55), (57, 57), (60, 60), (64, 64), (66, 66), (74, 75),
            (80, 83), (93, 95), (97, 99)]

In [67]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [68]:
def bert_weight_of_phrase(tokenized_text, ner_list):
    results = []
    for n in ner_list:
        weight_factor_sum = 0
        for i in range(n[0], n[1]+1):
            weight_factor = 100
            tokenized_text_copy = copy.copy(tokenized_text)
            tokenized_text_copy[i] = '[MASK]'
            # print(tokenized_text_copy)
            # print(tokenized_text_copy)
            # Convert token to vocabulary indices
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text_copy)

            # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
            segments_ids = [0]*len(tokenized_text_copy)

            # Convert inputs to PyTorch tensors
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])

            # Predict all tokens
            predictions = model(tokens_tensor, segments_tensors)

            item, pred = torch.sort(predictions[0, i], descending=True)
            predicted_tokens = []
            for index in pred[:100]:
                predicted_tokens.append(tokenizer.convert_ids_to_tokens([index.item()])[0])
            for one_token in predicted_tokens:
                if one_token != tokenized_text[i]:
                    weight_factor -= 1
                else:
                    break
            weight_factor_sum += weight_factor/100
        weight_factor_sum /= (n[1] - n[0] + 1)
        results.append([' '.join(tokenized_text[n[0]: n[1]+1]), weight_factor_sum])
    return results

In [69]:
def bert_weight_of_position(tokenized_text, ner_list):
    results = []
    for n in ner_list:
        weight_factor_sum = 0
        actual_range = [n[0], n[1]]
        if n[0] != 0:
            actual_range.insert(0, n[0]-1)
        if n[1] != len(ner_list):
            actual_range.append(n[0]+1)
        for i in actual_range:
            weight_factor = 100
            tokenized_text_copy = copy.copy(tokenized_text)
            tokenized_text_copy[i] = '[MASK]'
            # print(tokenized_text_copy)
            # print(tokenized_text_copy)
            # Convert token to vocabulary indices
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text_copy)

            # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
            segments_ids = [0]*len(tokenized_text_copy)

            # Convert inputs to PyTorch tensors
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])

            # Predict all tokens
            predictions = model(tokens_tensor, segments_tensors)

            item, pred = torch.sort(predictions[0, i], descending=True)
            predicted_tokens = []
            for index in pred[:100]:
                predicted_tokens.append(tokenizer.convert_ids_to_tokens([index.item()])[0])
            for one_token in predicted_tokens:
                if one_token != tokenized_text[i]:
                    weight_factor -= 1
                else:
                    break
            weight_factor_sum += weight_factor/100
        weight_factor_sum /= len(actual_range)
        results.append([' '.join(tokenized_text[n[0]: n[1]+1]), weight_factor_sum])
    return results

In [70]:
def average_weighing(weight_of_phrase, weight_of_position, k=0.5, m=0.5):
    results = []
    for one_entity_1, one_entity_2 in zip(weight_of_phrase, weight_of_position):
        weight = (one_entity_1[1]*k + one_entity_2[1]*m)/(k + m)
        results.append([one_entity_1[0], weight])
    return results

In [71]:
weight_of_phrase = bert_weight_of_phrase(tokenized_text, ner_list)
print(weight_of_phrase)

[['the united states of america', 0.998], ['50', 0.99], ['washington', 0.99], ['[UNK]', 1.0], ['the united states', 0.9966666666666667], ['five', 0.99], ['48', 0.9], ['washington', 1.0], ['[UNK]', 1.0], ['north america', 1.0], ['canada', 0.99], ['mexico', 0.97], ['two', 1.0], ['alaska', 1.0], ['hawaii', 1.0], ['north america', 1.0], ['the mid - pacific', 0.985], ['the pacific ocean', 0.9966666666666667], ['the caribbean sea', 1.0]]


In [72]:
weight_of_position = bert_weight_of_position(tokenized_text, ner_list)
print(weight_of_position)

[['the united states of america', 0.9966666666666667], ['50', 0.995], ['washington', 0.995], ['[UNK]', 1.0], ['the united states', 0.9975], ['five', 0.9275], ['48', 0.9199999999999999], ['washington', 0.9924999999999999], ['[UNK]', 0.9975], ['north america', 1.0], ['canada', 0.9924999999999999], ['mexico', 0.9824999999999999], ['two', 0.995], ['alaska', 1.0], ['hawaii', 1.0], ['north america', 1.0], ['the mid - pacific', 0.985], ['the pacific ocean', 0.9924999999999999], ['the caribbean sea', 1.0]]


In [73]:
weight_average = average_weighing(weight_of_phrase, weight_of_position)
print(weight_average)

[['the united states of america', 0.9973333333333334], ['50', 0.9924999999999999], ['washington', 0.9924999999999999], ['[UNK]', 1.0], ['the united states', 0.9970833333333333], ['five', 0.95875], ['48', 0.9099999999999999], ['washington', 0.99625], ['[UNK]', 0.99875], ['north america', 1.0], ['canada', 0.99125], ['mexico', 0.97625], ['two', 0.9975], ['alaska', 1.0], ['hawaii', 1.0], ['north america', 1.0], ['the mid - pacific', 0.985], ['the pacific ocean', 0.9945833333333334], ['the caribbean sea', 1.0]]


In [84]:
def factorizer(data):
    result = []
    for x in data:
        result.append([x[0], round((math.pow(101, x[1]) - 1), 1)])
    return result

In [85]:
weight_of_phrase_factorized = factorizer(weight_of_phrase)
print(weight_of_phrase_factorized)

[['the united states of america', 99.1], ['50', 95.4], ['washington', 95.4], ['[UNK]', 100.0], ['the united states', 98.5], ['five', 95.4], ['48', 62.7], ['washington', 100.0], ['[UNK]', 100.0], ['north america', 100.0], ['canada', 95.4], ['mexico', 86.9], ['two', 100.0], ['alaska', 100.0], ['hawaii', 100.0], ['north america', 100.0], ['the mid - pacific', 93.2], ['the pacific ocean', 98.5], ['the caribbean sea', 100.0]]


In [86]:
weight_of_position_factorized = factorizer(weight_of_position)
print(weight_of_position_factorized)

[['the united states of america', 98.5], ['50', 97.7], ['washington', 97.7], ['[UNK]', 100.0], ['the united states', 98.8], ['five', 71.3], ['48', 68.8], ['washington', 96.6], ['[UNK]', 98.8], ['north america', 100.0], ['canada', 96.6], ['mexico', 92.2], ['two', 97.7], ['alaska', 100.0], ['hawaii', 100.0], ['north america', 100.0], ['the mid - pacific', 93.2], ['the pacific ocean', 96.6], ['the caribbean sea', 100.0]]


In [88]:
weight_average_factorized = factorizer(weight_average)
print(weight_average_factorized)

[['the united states of america', 98.8], ['50', 96.6], ['washington', 96.6], ['[UNK]', 100.0], ['the united states', 98.6], ['five', 82.5], ['48', 65.7], ['washington', 98.3], ['[UNK]', 99.4], ['north america', 100.0], ['canada', 96.0], ['mexico', 89.5], ['two', 98.8], ['alaska', 100.0], ['hawaii', 100.0], ['north america', 100.0], ['the mid - pacific', 93.2], ['the pacific ocean', 97.5], ['the caribbean sea', 100.0]]
